In [47]:
import urllib3
import urllib3.request


import tweepy
import csv
import json
from pandas.io.json import json_normalize


import pandas as pd
import numpy as np

In [48]:
####input your credentials here
consumer_key = '4ha4rLgP6Ci6fEZtaqttGTKoA'
consumer_secret = '5ckLaCgfTdfmWM7qS9f2w05pDCSIWRCTHlm7RLnKwK9tCWIz9P'
access_token = '602145669-jHmxtsl0wSZDFeZxi81GcTzYrD87dRBhF78ip0qo'
access_token_secret = 'YFLMmVVdcN4gb4KDX3MeOjbjxoKnnsFvjKxjRGMkkEZ5D'

In [49]:
input_hashtag = "giletjaune"
n_tweets = 1
language = "fr"
since_date = "2018-03-07"

In [50]:
####request to Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify = True)

inc = 0
for tweet in tweepy.Cursor(api.search,q="#"+input_hashtag,#count=n_tweets,
                           lang=language,
                           since=since_date).items():
    inc = +1
    with open('tweets/tweet'+str(tweet.id)+'.json', 'w', encoding='utf8') as file:
        ####you need to create a 'tweets' folder
        
        tweet_json = tweet._json
        json.dump(tweet_json, file)
        

Rate limit reached. Sleeping for: 839


KeyboardInterrupt: 

In [51]:
##### test file opened in the notebook
with open('tweets/tweet1106246365007527937.json') as json_file:  
    tweet_test = json.load(json_file)
    

In [52]:
def is_a_retweet(tweet):
    #### returns a boolean 
    return tweet.get('retweeted_status',None) != None

In [132]:
#initialisation of databases, df files

d_users = {'user_id': [],
           'name':[],
           'screen_name':[],
           'location':[],
           'description':[],
           'url':[],
           'followers_count':[],
           'friends_count':[],
           'listed_count':[],
           'created_at':[],
           'favourites_count':[],
           'geo_enabled':[],
           'verified':[],
           'statuses_count':[],
           'lang':[],
           'contributors_enabled':[]
          }
df_users = pd.DataFrame(data=d_users)

d_tweets = {'tweet_id': [],
            'created_at': [],
            'text': [],
            'truncated': [],
            'source': [],
            'in_reply_to_status_id': [],
            'in_reply_to_user_id': [],
            'in_reply_to_screen_name': [],
            'user_id': [],
            'geo': [],
            'coordinates': [],
            'place': [],
            'contributors': [],
            'is_quote_status': [],
            'retweet_count': [],
            'favorite_count': [],
            'favorited': [],
            'retweeted': [],
            'lang': []         
           }
df_tweets = pd.DataFrame(data=d_tweets)


d_hashtags={'hashtag': [], 'tweet_id': []}
df_hashtags = pd.DataFrame(data=d_hastags)

d_users_mentions = {'tweet_id':[], 'user_id': [], 'screen_name':[], 'name':[]}
df_users_mentions = pd.DataFrame(data=d_users_mentions)

d_retweet_users = {'user_id':[], 'original_user_id':[], 'original_tweet_id':[]}
df_retweet_users= pd.DataFrame(data=d_retweet_users)

d_influent_users = {'user_id':[], 
                    'name':[],
                    'retweet_count':[],
                    'favorite_count':[], 
                    'description':[] 
                   }

df_influent_users = pd.DataFrame(data=d_influent_users)



In [130]:
def store_hashtag(tweet):
    global input_hashtag
    global df_hashtags
    if tweet['entities']['hashtags']==[]:
        df_hashtags = df_hashtags.append({'hashtag': input_hashtag ,'tweet_id':tweet['id_str']}, ignore_index=True)
    else:
        for raw_hash in tweet['entities']['hashtags']:
            df_hashtags = df_hashtags.append({'hashtag': raw_hash['text'] ,'tweet_id':tweet['id_str']}, ignore_index=True)

def store_user_mentions(tweet):
    global df_users_mentions
    tweet_id = tweet['id_str']
    for raw_mention in tweet['entities']['user_mentions']:
        screen_name = raw_mention['screen_name']
        name = raw_mention['name']
        user_id = raw_mention['id_str']
        df_users_mentions = df_users_mentions.append({'tweet_id':tweet_id, 'user_id': user_id, 'screen_name':screen_name, 'name':name},ignore_index=True)

def store_users(tweet):
    global df_users
    user = tweet['user']
    d_user = {'user_id': user['id_str'],
              'name':user['name'],
              'screen_name':user['screen_name'],
              'location':user['location'],
              'description':user['description'],
              'url':user['url'],
              'followers_count':user['followers_count'],
              'friends_count':user['friends_count'],
              'listed_count':user['listed_count'],
              'created_at':user['created_at'],
              'favourites_count':user['favourites_count'],
              'geo_enabled':user['geo_enabled'],
              'verified':user['verified'],
              'statuses_count':user['statuses_count'],
              'lang':user['lang'],
              'contributors_enabled':user['contributors_enabled']
             }
    df_users = df_users.append(d_user,ignore_index=True)
    
def store_tweet(tweet):
    global df_tweets
    if is_a_retweet(tweet):
        store_tweet(tweet['retweeted_status'])
    else:
        df_tweets = df_tweets.append({'tweet_id':  tweet['id_str'],
            'created_at': tweet['created_at'],
            'text': tweet['text'],
            'truncated': tweet['truncated'],
            'source': tweet['source'],
            'in_reply_to_status_id': tweet['in_reply_to_status_id_str'],
            'in_reply_to_user_id': tweet['in_reply_to_user_id_str'],
            'in_reply_to_screen_name': tweet['in_reply_to_screen_name'],
            'user_id': tweet['user']['id_str'],
            'geo': tweet['geo'],
            'coordinates': tweet['coordinates'],
            'place': tweet['place'],
            'contributors': tweet['contributors'],
            'is_quote_status': tweet['is_quote_status'],
            'retweet_count': tweet['retweet_count'],
            'favorite_count': tweet['favorite_count'],
            'favorited': tweet['favorited'],
            'retweeted': tweet['retweeted'],
            'lang': tweet['lang'] },ignore_index=True)
        
def store_retweet_user(tweet):
    global df_retweet_users
    if is_a_retweet(tweet):
        df_retweet_users = df_retweet_users.append({'user_id':tweet['user']['id_str'],
                                                    'original_user_id':tweet['retweeted_status']['user']['id_str'],
                                                    'original_tweet_id':tweet['retweeted_status']['id_str']}, ignore_index=True)
        
def store_influent_users(n):
    #n is the number of influent users return
    df_favorite_count = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).head()
    for i in range(n):
        d_influent_users = {'user_id':df_favorite_count[0], 
                            'name':[],
                            'retweet_count':[],
                            'favorite_count':[], 
                            'description':[] 
                           }
        

In [135]:
#### creation of the databases

import os
path_to_json = 'tweets/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for json_file in json_files:
    
    with open("tweets/"+json_file) as tweet:
        tweet = json.load(tweet)
        store_hashtag(tweet)
        store_user_mentions(tweet)
        store_users(tweet)
        store_tweet(tweet)
        store_retweet_user(tweet)
        
        

# Beginnig of agregations

In [60]:
df_favorite_count = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_retweet_count = df_tweets.groupby(['user_id'])['retweet_count'].agg('sum').sort_values(ascending=False).reset_index()

df_favorite_count = df_favorite_count.merge(df_retweet_count, how = 'left', on = "user_id")
df_favorite_count = df_favorite_count.merge(df_users[["user_id","screen_name","name","description"]].drop_duplicates(), how = 'left', on = 'user_id')


In [21]:
df_tweets

,tweet_id,created_at,text,truncated,source,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,user_id,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang
0,1105895180341837824,Wed Mar 13 18:15:32 +0000 2019,"Parfaitement résumé , \n Quel clarté,je valide...",1.0,"<a href=""http://twitter.com/download/android"" ...",None,None,None,2569469946,None,None,None,None,0.0,8.0,15.0,0.0,0.0,fr
1,1106185627102527488,Thu Mar 14 13:29:40 +0000 2019,"82 ans ... ""On souhaite que ce gouvernement et...",1.0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,None,None,1028668294751039488,None,None,None,None,0.0,954.0,1377.0,0.0,0.0,fr
2,1101779937164492801,Sat Mar 02 09:43:01 +0000 2019,Samedi #GiletsJaunes #GiletJaune Toujours Oppo...,1.0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,None,None,2889517437,None,None,None,None,0.0,3.0,3.0,0.0,0.0,fr
3,1105187889472786437,Mon Mar 11 19:25:00 +0000 2019,Les chiens de garde de la caste. \n#ActeXVIII ...,0.0,"<a href=""http://twitter.com/download/iphone"" r...",None,None,None,2411032473,None,None,None,None,0.0,10.0,12.0,0.0,0.0,fr
4,1106065402789052416,Thu Mar 14 05:31:56 +0000 2019,@PureTele Quand cessera ce mépris en vers l'en...,1.0,"<a href=""http://twitter.com/download/android"" ...",1105862107013234688,1068532568608464896,PureTele,1064870090028527616,None,None,None,None,0.0,3.0,9.0,0.0,0.0,fr
5,1106185627102527488,Thu Mar 14 13:29:40 +0000 2019,"82 ans ... ""On souhaite que ce gouvernement et...",1.0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,None,None,1028668294751039488,None,None,None,None,0.0,954.0,1377.0,0.0,0.0,fr
6,1106185627102527488,Thu Mar 14 13:29:40 +0000 2019,"82 ans ... ""On souhaite que ce gouvernement et...",1.0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,None,None,1028668294751039488,None,None,None,None,0.0,954.0,1377.0,0.0,0.0,fr
7,1103347304445161473,Wed Mar 06 17:31:11 +0000 2019,#24hpujadas\n Incroyable à #Nantes #GiletsJaun...,1.0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,None,None,435148006,None,None,None,None,0.0,806.0,381.0,0.0,0.0,fr
8,1106185627102527488,Thu Mar 14 13:29:40 +0000 2019,"82 ans ... ""On souhaite que ce gouvernement et...",1.0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,None,None,1028668294751039488,None,None,None,None,0.0,954.0,1377.0,0.0,0.0,fr
9,1106185627102527488,Thu Mar 14 13:29:40 +0000 2019,"82 ans ... ""On souhaite que ce gouvernement et...",1.0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",None,None,None,1028668294751039488,None,None,None,None,0.0,954.0,1377.0,0.0,0.0,fr


In [61]:
df_tweets['text'].iloc[0]

'Parfaitement résumé , \n Quel clarté,je valide totalement 🏆🏆🏆🏆🏆\n#giletjaune #reseauxsociaux \n#media #Cnews #BFM… https://t.co/Bx5zEnxFL6'

In [120]:
df_hashtags


,hashtag,tweet_id
0,"{'text': 'giletjaune', 'indices': [82, 93]}",1105910660867768320
1,"{'text': 'reseauxsociaux', 'indices': [94, 109]}",1105910660867768320
2,"{'text': 'media', 'indices': [111, 117]}",1105910660867768320
3,"{'text': 'Cnews', 'indices': [118, 124]}",1105910660867768320
4,"{'text': 'BFM', 'indices': [125, 129]}",1105910660867768320
5,"{'text': 'BFMTV', 'indices': [130, 136]}",1105910660867768320
6,giletjaune,1106253587519291393
7,"{'text': 'GiletsJaunes', 'indices': [27, 40]}",1106468282771693568
8,"{'text': 'GiletJaune', 'indices': [41, 52]}",1106468282771693568
9,"{'text': 'bobo', 'indices': [99, 104]}",1106468282771693568


In [310]:
df_favorite_count = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).to_frame()
df_retweet_count = df_tweets.groupby(['user_id'])['retweet_count'].agg('sum').sort_values(ascending=False).to_frame()


In [320]:
df_favorite_count.index[17]


'514602810'

In [63]:
df_hashtags.set_index(["", ""]).count(level="hashtag")
df_hashtags_txt

TypeError: Can only count levels on hierarchical index.

In [116]:
d_hashtags_bis = {'hashtag':[]}
df_hashtags_bis = pd.DataFrame(data=d_hashtags_bis)

for i in range(len(df_hashtags)):
    if df_hashtags['hashtag'][i]!=input_hashtag:
        
        h = df_hashtags['hashtag'][i]['text']
        print(h)
        df_hashtags_bis.append({'hashtag':h},ignore_index=True)
    


TypeError: Can only append a Series if ignore_index=True or if the Series has a name

In [117]:
df_hashtags

,hashtag,tweet_id
0,"{'text': 'giletjaune', 'indices': [82, 93]}",1105910660867768320
1,"{'text': 'reseauxsociaux', 'indices': [94, 109]}",1105910660867768320
2,"{'text': 'media', 'indices': [111, 117]}",1105910660867768320
3,"{'text': 'Cnews', 'indices': [118, 124]}",1105910660867768320
4,"{'text': 'BFM', 'indices': [125, 129]}",1105910660867768320
5,"{'text': 'BFMTV', 'indices': [130, 136]}",1105910660867768320
6,giletjaune,1106253587519291393
7,"{'text': 'GiletsJaunes', 'indices': [27, 40]}",1106468282771693568
8,"{'text': 'GiletJaune', 'indices': [41, 52]}",1106468282771693568
9,"{'text': 'bobo', 'indices': [99, 104]}",1106468282771693568


In [83]:
len(df_hashtags)

3786

In [90]:
df_hashtags['hashtag'][100]

'giletjaune'

In [108]:
h = df_hashtags['hashtag'][89]['text']

In [109]:
h

'France2'

In [126]:
df_hashtags.count(axis='hashtag')

ValueError: No axis named hashtag for object type <class 'pandas.core.frame.DataFrame'>

In [136]:
tweet_agg = df_hashtags.groupby(['hashtag']).size().sort_values(ascending=False)

In [170]:


df_emergency = tweet_agg.to_frame()
df_emergency.to_csv('emergency.csv')

In [134]:
d_hashtags

{'hashtag': [], 'tweet_id': []}

In [ ]:
df_tweet_with_hashtags = df_tweets.join(df_hashtags)

In [146]:
best_user_liked = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False)
best_user_rt = df_tweets.groupby(['user_id'])['retweet_count'].agg('sum').sort_values(ascending=False)

In [147]:
print(best_user_liked.head(10))
print(best_user_rt.head(10))

user_id
1028668294751039488    1027415.0
1071436971191492609     102098.0
3110034796               16044.0
1069545278968078338      12740.0
155606403                12488.0
1028639615811510274      10977.0
472852289                 8123.0
820545530359119873        4615.0
1068532568608464896       4308.0
435148006                 3810.0
Name: favorite_count, dtype: float64
user_id
1028668294751039488    718032.0
1071436971191492609    189206.0
155606403               19062.0
472852289               15037.0
1069545278968078338     11346.0
435148006                8060.0
1068532568608464896      6270.0
1028639615811510274      5807.0
3110034796               5736.0
1024740309618577408      2622.0
Name: retweet_count, dtype: float64


In [162]:
df_best_liked = best_user_liked.to_frame()
print(type(df_best_liked))
print(df_best_liked.reset_index().columns)

<class 'pandas.core.frame.DataFrame'>
Index(['user_id', 'favorite_count'], dtype='object')


In [163]:
#df_users_process = df_users[df_users['user_id'] == '1028668294751039488']
df_best_liked = best_user_liked.to_frame().reset_index().set_index('user_id')
df_users_process = df_users.set_index('user_id').join(df_best_liked)

In [264]:
top_influencers = df_users_process.drop_duplicates().sort_values('favorite_count', ascending = False)
top_influencers.columns

Index(['name', 'screen_name', 'location', 'description', 'url',
       'followers_count', 'friends_count', 'listed_count', 'created_at',
       'favourites_count', 'geo_enabled', 'verified', 'statuses_count', 'lang',
       'contributors_enabled', 'favorite_count'],
      dtype='object')

In [271]:
pd.options.display.max_colwidth = 200
pd.options.display.precision = 0

In [266]:
top_influencers = top_influencers[['name','description', 'followers_count', 'friends_count','favorite_count']]
top_influencers = top_influencers.reset_index().drop(columns = ["user_id"])

In [267]:
top_influencers.columns = ["Name","Description","Number of followers","Number of friends","Number of favorites"]

In [273]:
top_influencers = top_influencers.head(10)
top_influencers["Number of favorites"] = top_influencers["Number of favorites"].apply(int)
top_influencers["Number of followers"] = top_influencers["Number of followers"].apply(int)
top_influencers["Number of friends"] = top_influencers["Number of friends"].apply(int)


In [274]:
top_influencers.style.set_properties(**{'text-align': 'left'})

,Name,Description,Number of followers,Number of friends,Number of favorites
0,LesGiletsJaunes.fr,Mouvement citoyen apartisan pour une démocratie d'avenir. #GiletJaune #GiletsJaunes #RIC,28518,0,1027415
1,MichaelVllt 🇫🇷,Rebel with many causes. https://t.co/ISNLNObWbX,7338,991,12488
2,retraitman #LFI,"Forçat de la route en retraite, syndicaliste CGT. On a eut #SarkoLeVoyou, #HollandeSuperMenteur et maintenant #MacronPiègeACons ça va s'arrêter quant la mouise?",2717,2191,3810
3,Alexandre macron matricule 007🔫,,230,66,1926
4,de Cabarrus Thierry,"Après 44 ans de journalisme ""objectif"" (carte de presse N°35183), je reprends ma liberté et j'abandonne une neutralité confortable. Ex chroniqueur sur Le Plus",7983,1042,909
5,Mike Bresson #FBPE 🇪🇺🇫🇷 #26Mai,"Adjoint au Maire, conseiller communautaire, 1er Vice-président du SCOT #StopGiletsJaunes",3803,1595,582
6,yellow defender,Patriote et GILET JAUNE. Antin macron,21,20,275
7,LCI,"Bienvenue sur LCI, #La26 sur la TNT. Le meilleur de l'info, de l'analyse, du décryptage et tous les grands événements en direct #RespectZone",151892,994,247
8,SMAGMA,Collectif d'humoristes · 3 garçons dans le vent 👨‍👨‍👦 · Une nouvelle vidéo un vendredi sur deux sur YouTube 📺 · D'autres bêtises sur Facebook et Instagram !,1,0,169
9,Ptit-eOurs-eBlanc-he,PtitOursBlanc (rien à voir avec les fachos!) Pr les curieux : https://t.co/46wHWjKdyF… #injustices #AntiHaine #LaPlaneteVaMal,346,795,135


In [269]:
top_influencers

,Name,Description,Number of followers,Number of friends,Number of favorites
0,LesGiletsJaunes.fr,Mouvement citoyen apartisan pour une démocratie d'avenir. #GiletJaune #GiletsJaunes #RIC,28518,0,1027415
1,MichaelVllt 🇫🇷,Rebel with many causes. https://t.co/ISNLNObWbX,7338,991,12488
2,retraitman #LFI,"Forçat de la route en retraite, syndicaliste CGT.\nOn a eut #SarkoLeVoyou, #HollandeSuperMenteur et maintenant #MacronPiègeACons\nça va s'arrêter quant la mouise?",2717,2191,3810
3,Alexandre macron matricule 007🔫,,230,66,1926
4,de Cabarrus Thierry,"Après 44 ans de journalisme ""objectif"" (carte de presse N°35183), je reprends ma liberté et j'abandonne une neutralité confortable. Ex chroniqueur sur Le Plus",7983,1042,909
5,Mike Bresson #FBPE 🇪🇺🇫🇷 #26Mai,"Adjoint au Maire, conseiller communautaire, 1er Vice-président du SCOT #StopGiletsJaunes",3803,1595,582
6,yellow defender,Patriote et \nGILET JAUNE.\nAntin macron,21,20,275
7,LCI,"Bienvenue sur LCI, #La26 sur la TNT. Le meilleur de l'info, de l'analyse, du décryptage et tous les grands événements en direct #RespectZone",151892,994,247
8,SMAGMA,Collectif d'humoristes · 3 garçons dans le vent 👨‍👨‍👦 · Une nouvelle vidéo un vendredi sur deux sur YouTube 📺 · D'autres bêtises sur Facebook et Instagram !,1,0,169
9,Ptit-eOurs-eBlanc-he,PtitOursBlanc (rien à voir avec les fachos!)\nPr les curieux : \nhttps://t.co/46wHWjKdyF…\n\n#injustices #AntiHaine #LaPlaneteVaMal,346,795,135


In [263]:
top_influencers.to_html('topinfluencers.html',index=False)

In [181]:
df_tweets_hashtags = df_hashtags.set_index('tweet_id').join(df_tweets.set_index('tweet_id'))

In [185]:
df_tw_hash_user = df_tweets_hashtags.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))

In [203]:
df_aggretated = df_tw_hash_user.groupby(['user_id','hashtag']).size().reset_index()
print(type(df_aggretated))

<class 'pandas.core.frame.DataFrame'>


In [207]:
df_aggretated[df_aggretated['user_id'] == '936255936854921217']

,user_id,hashtag,0
522,936255936854921217,GiletJaune,3
523,936255936854921217,bfm,3
524,936255936854921217,btp,3
525,936255936854921217,cnews,3
526,936255936854921217,giletsjaunes,3
527,936255936854921217,tpmp,3
